In [55]:
import warnings
warnings.filterwarnings('ignore')

import os
import re
import json
import webbrowser
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider
from IPython.display import display, Javascript, Markdown, clear_output, HTML
from ipywidgets import GridspecLayout
from markdown_it import MarkdownIt

In [56]:
%run ../02_ui_components/progressbar.ipynb

In [57]:
%run ../02_ui_components/sidebar.ipynb

In [58]:
%run ../02_ui_components/dropdown.ipynb

In [59]:
%run ../02_ui_components/header_button.ipynb

In [60]:
%run ../02_ui_components/more_info.ipynb

In [61]:
# Placeholder variables to store paths and data
# ques_nav_path = None
back_list = None
stored_choices = None
footer = None
path = None
back_path = None

In [62]:
# Declaring necessary variables and importing neccesary files for storing user answers and keep track of the visited pages
global stored_choices, path, back_path, ques_nav_path, back_list

# stored_choices = {}
# path = "../data_files/json/temp_storage.json"
# back_path = "../data_files/json/backtracking.json"
dropdown_path = "../data_files/csv/links.csv"
ques_nav_path = "../data_files/json/ques_nav_list.json"

In [63]:
# # Importing the json file for storing user answers
# with open(path, "r") as f:
#     stored_choices = json.load(f)

In [64]:
# # Importing the json file for keeping track of visited pages
# with open(back_path, 'r') as f:
#     back_list = json.load(f)

In [65]:
# Creating button
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto', border='1px solid black')) #width='max-content'

In [66]:
def get_css_class(css_file, class_name):
    with open(css_file, 'r') as file:
        css = file.read()
        # Use regex to extract the class
        pattern = r'\.' + re.escape(class_name) + r'\s*\{([^}]*)\}'
        match = re.search(pattern, css, re.DOTALL)
        if match:
            css_content = match.group(1).strip()
            return f'\n<style>\n.{class_name} {{\n{css_content}\n}}\n</style>\n'
        else:
            return None

In [67]:
css_path = "../css/style.css"

In [68]:
# Custom CSS to modify the question button design
ques_button_css = get_css_class(css_path, 'ques-button')

In [69]:
# Custom CSS to modify the option button design
opt_button_css = get_css_class(css_path, 'opt-button')

In [70]:
# Custom CSS to modify the header design
header_css = get_css_class(css_path, 'custom-header')

In [71]:
# Custom CSS to modify the footer design
footer_css = get_css_class(css_path, 'custom-footer')

In [72]:
# Create an HTML widget for the background color
background_color_html = widgets.HTML(
    value = get_css_class(css_path, 'custom-vbox')
)

In [73]:
# Taking from separate header notebook
header = header_prog1
header_widget = header_widget

In [74]:
file_name = widgets.Text(
    value='',
    placeholder='Paste your copied string here',
    description='String:',
    disabled=False   
)

In [75]:
# Create a buttons for submitting the file name
file_name_submit_button = widgets.Button(description="Submit", layout=widgets.Layout(border='1px solid black'))

In [76]:
# Creating a container for header image
header_right = widgets.VBox([file_name, file_name_submit_button], 
                    layout=widgets.Layout(width='auto', 
                                          padding_top= '10px', 
                                          align_items='flex-end',
                                          justify_content='center'))

In [79]:
footer_output = widgets.Output()
def on_file_button_clicked(event):
    with footer_output:
        clear_output()
        global ques_nav_path, back_list, stored_choices, path, back_path, footer
        
        main_file_name = file_name.value
        path = f"../data_files/json/{main_file_name}_storage.json"
        back_path = f"../data_files/json/{main_file_name}_back_path.json"
        ques_nav_path = f"../data_files/json/{main_file_name}_ques_nav_path.json"
        if not os.path.isfile(path):
            with open(path, 'w') as file:
                json.dump({}, file)
        
        if not os.path.isfile(back_path):
            with open(back_path, 'w') as file:
                json.dump([], file)
        data_content = {
            "obj_list": [["Select a notebook", None]],
            "st_area_list": [["Select a notebook", None]],
            "dt_list": [["Select a notebook", None]],
            "ts_list": [["Select a notebook", None]],
            "ed_list": [["Select a notebook", None]],
            "da_list": [["Select a notebook", None]],
            "rec_list": [["Select a notebook", None]]
        }
        if not os.path.isfile(ques_nav_path):
            with open(ques_nav_path, 'w') as file:
                json.dump(data_content, file, indent=2)
                
        # Importing the json file for storing user answers
        with open(path, "r") as f:
            stored_choices = json.load(f)
            
        # Importing the json file for keeping track of visited pages
        with open(back_path, 'r') as f:
            back_list = json.load(f)
            
        notebook_name = "objective"
        notebook_list_name = "obj_list"
        footer_name = "footer_in_prog1"
        
        footer = main(notebook_name, notebook_list_name, footer_name)
#         file_footer = widgets.VBox(children=[footer], layout=widgets.Layout(
#                 width='88%',  #, popup
#                 height='100%',  
#                 border_left='solid 1px grey',
#                 padding='5px',
#                 margin='auto',  
#                 align_items='center',  
#                 justify_content='center'
#         ))
        
        display(footer)
file_name_submit_button.on_click(on_file_button_clicked)

In [80]:
# Creating a container for header image
header_container = widgets.HBox([header, header_widget, header_right], 
                    layout=widgets.Layout(width='95%', 
                                          padding_top= '10px', 
                                          align_items='flex-start',
                                          justify_content='space-between'))

In [81]:
# notebook_name = "objective"
# notebook_list_name = "obj_list"
# footer_name = "footer_in_prog1"

In [82]:
# footer = main(notebook_name, notebook_list_name, footer_name)

In [83]:
# Reading the CSV data file into a DataFrame
df = pd.read_csv('../data_files/csv/question_data.csv', low_memory=False)

In [84]:
dp_df = pd.read_csv(dropdown_path, low_memory=False)

In [85]:
drop_link = dp_df.loc[dp_df['field_code'] == 'objective', 'info_url'].iloc[0]

In [86]:
# Assigns dynamic link to the last sidebar button based on the notebook dropdown info link
concept_grid[5, 0:9].on_click(on_concept_option_clicked(drop_link))

In [87]:
# seperates the string based on ',' also removes the whitespace after comma & makes a list
def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [88]:
# gets the question based on value of other column
ques = df.loc[df['question_code'] == 'objective', 'question_text'].iloc[0]
opt = df.loc[df['question_code'] == 'objective', 'q_option_label'].iloc[0]
opt_codes = df.loc[df['question_code'] == 'objective', 'q_option_code'].iloc[0]

# Using the split_and_trim function for neccessary steps
opt = split_and_trim(opt)
opt_codes = split_and_trim(opt_codes)

In [89]:
html_widget = more_info_widget("../data_files/markdown/index.md")

In [90]:
# Factor widget
question = create_expanded_button(ques, 'success')
question.style.button_color = 'lightblue'
question.style.text_color = 'black'
# question.style.font_weight = 'bold'

# Add custom class to the button
question.add_class('ques-button') # Adding custom css properties to the buttons

option1 = create_expanded_button(opt[0], 'success')
option1.style.button_color = '#DAE8FC'
option1.style.text_color = 'black'
option1.add_class('opt-button')

option2 = create_expanded_button(opt[1], 'success')
option2.style.button_color = '#DAE8FC'
option2.style.text_color = 'black'
option2.add_class('opt-button')

option3 = create_expanded_button(opt[2], 'success')
option3.style.button_color = '#DAE8FC'
option3.style.text_color = 'black'
option3.add_class('opt-button')

option4 = create_expanded_button(opt[3], 'success')
option4.style.button_color = '#DAE8FC'
option4.style.text_color = 'black'
option4.add_class('opt-button')

option5 = create_expanded_button(opt[4], 'success')
option5.style.button_color = '#DAE8FC'
option5.style.text_color = 'red'
option5.add_class('opt-button')

option6 = create_expanded_button(opt[5], 'success')
option6.style.button_color = '#DAE8FC'
option6.style.text_color = 'red'
option6.add_class('opt-button')

option7 = create_expanded_button(opt[6], 'success')
option7.style.button_color = '#DAE8FC'
option7.style.text_color = 'red'
option7.add_class('opt-button')

option8 = create_expanded_button(opt[7], 'success')
option8.style.button_color = '#DAE8FC'
option8.style.text_color = 'black'
option8.add_class('opt-button')

option9 = create_expanded_button(opt[8], 'success')
option9.style.button_color = '#F5F5F5'
option9.style.text_color = 'black'
option9.add_class('opt-button')

# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=ques_button_css)
css_widget2 = widgets.HTML(value=opt_button_css)

factor_grid = GridspecLayout(10, 18, grid_gap='5px', height='550px')
factor_grid[4:8, 1:7] = question
factor_grid[1:8, 13:17] = html_widget
factor_grid[1, 8:12] = option1
factor_grid[2, 8:12] = option2
factor_grid[3, 8:12] = option3
factor_grid[4, 8:12] = option4
factor_grid[5, 8:12] = option5
factor_grid[6, 8:12] = option6
factor_grid[7, 8:12] = option7
factor_grid[8, 8:12] = option8
factor_grid[9, 8:12] = option9
# factor_grid[10, 7:9] = option10

# Creating a container to store the factor widget and added custom css widgets
logical_integer_container = widgets.VBox(children=[factor_grid, css_widget, css_widget2], layout=widgets.Layout(
    align_items='center'  # Center align vertically
#     border='solid 2px black',
#     width = "70%"
#     justify_content='center'  # Center align horizontally
#     height='100vh'  # Set height to 100% of viewport height for vertical centering
))

In [91]:
# Creating button list for implementing the checkmark feature
button_list = [option1, option2, option3, option4, option5, option6, option7, option8, option9]

In [92]:
selected = False

In [93]:
# Create the text widget with a warning message
warning_widget2 = widgets.HTML(
    value="""
    <div style="padding: 5px; background-color: #f8d7da; color: #721c24; border: 2px solid #f5c6cb; border-radius: 5px;">
        <strong>Warning!</strong> Unavailable for now. Please select another.
    </div>
    """
)

In [94]:
error2 = widgets.HBox(
    [warning_widget2],
    layout=widgets.Layout(
        width='auto',  
        height='auto',
        margin='auto',  
        align_items='flex-end',  
        justify_content='flex-end' 
    )
)

In [95]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_1_button_clicked(option1):
    with output:
        clear_output()
        global selected, stored_choices
        
        selected = True
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option1:
                btn.icon='check'
            else:
                btn.icon=''
        
        # storing all the selected option information
        selected_option = factor_grid[1, 8:12].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_code
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
        
# Calling the on click function
factor_grid[1, 8:12].on_click(option_1_button_clicked)

In [96]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_2_button_clicked(option2):
    with output:
        clear_output()
        global selected
        
        selected = True
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option2:
                btn.icon='check'
            else:
                btn.icon=''

        # storing all the selected option information
        selected_option = factor_grid[2, 8:12].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_code
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function        
factor_grid[2, 8:12].on_click(option_2_button_clicked)

In [97]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_3_button_clicked(option3):
    with output:
        clear_output()
        global selected
        
        selected = True
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option3:
                btn.icon='check'
            else:
                btn.icon=''
        
        # storing all the selected option information
        selected_option = factor_grid[3, 8:12].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_code
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function       
factor_grid[3, 8:12].on_click(option_3_button_clicked)

In [98]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_4_button_clicked(option4):
    with output:
        clear_output()
        global selected
        
        selected = True
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option4:
                btn.icon='check'
            else:
                btn.icon=''
        
        # storing all the selected option information
        selected_option = factor_grid[4, 8:12].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_code
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function       
factor_grid[4, 8:12].on_click(option_4_button_clicked)

In [99]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_5_button_clicked(option5):
    with output:
        clear_output()
        global selected
        
        selected = False
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option5:
                btn.icon='check'
            else:
                btn.icon=''
        display(error2)
        
        #### DISABLIING OPTIONS FOR NOW
#         # storing all the selected option information
#         selected_option = factor_grid[5, 8:12].description
#         selected_option_index = opt.index(selected_option)
#         selected_option_code = opt_codes[selected_option_index]
        
#         # Retrieving infomation for using inside logic while determining the next page
#         stored_choices["objective"] = selected_option_code
        
#         # Storing all the information
#         with open(path, "w") as f:
#             json.dump(stored_choices, f)
            
# Calling the on click function       
factor_grid[5, 8:12].on_click(option_5_button_clicked)

In [100]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_6_button_clicked(option6):
    with output:
        clear_output()
        global selected
        
        selected = False
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option6:
                btn.icon='check'
            else:
                btn.icon=''
        display(error2)
         #### DISABLIING OPTIONS FOR NOW
#         # storing all the selected option information
#         selected_option = factor_grid[6, 8:12].description
#         selected_option_index = opt.index(selected_option)
#         selected_option_code = opt_codes[selected_option_index]
        
#         # Retrieving infomation for using inside logic while determining the next page
#         stored_choices["objective"] = selected_option_code
        
#         # Storing all the information
#         with open(path, "w") as f:
#             json.dump(stored_choices, f)
            
# Calling the on click function     
factor_grid[6, 8:12].on_click(option_6_button_clicked)

In [101]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_7_button_clicked(option7):
    with output:
        clear_output()
        global selected
        
        selected = False
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option7:
                btn.icon='check'
            else:
                btn.icon=''
                
        display(error2)       
                
         #### DISABLIING OPTIONS FOR NOW
#         # storing all the selected option information
#         selected_option = factor_grid[7, 8:12].description
#         selected_option_index = opt.index(selected_option)
#         selected_option_code = opt_codes[selected_option_index]
        
#         # Retrieving infomation for using inside logic while determining the next page
#         stored_choices["objective"] = selected_option_code
        
#         # Storing all the information
#         with open(path, "w") as f:
#             json.dump(stored_choices, f)



            
# Calling the on click function     
factor_grid[7, 8:12].on_click(option_7_button_clicked)

In [102]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_8_button_clicked(option8):
    with output:
        clear_output()
        global selected
        
        selected = True
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option8:
                btn.icon='check'
            else:
                btn.icon=''
        
        # storing all the selected option information
        selected_option = factor_grid[8, 8:12].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_code
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function       
factor_grid[8, 8:12].on_click(option_8_button_clicked)

In [103]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_9_button_clicked(option9):
    with output:
        clear_output()
        global selected
        
        selected = True
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option9:
                btn.icon='check'
            else:
                btn.icon=''
        
        # storing all the selected option information
        selected_option = factor_grid[9, 8:12].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_code
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function      
factor_grid[9, 8:12].on_click(option_9_button_clicked)

In [104]:
# button.on_click(show_popup(None, drop_link))
button.on_click(lambda b: show_popup(b, drop_link))

In [105]:
close_button.on_click(close_popup)

In [106]:
# Create a buttons for submitting the question
submit_button = widgets.Button(description="Submit & Next >>", layout=widgets.Layout(border='1px solid black'))

In [107]:
# Create a button for going back to previous question
back_button = widgets.Button(description="<< Back", layout=widgets.Layout(border='1px solid black'))

In [108]:
# Create the text widget with a warning message
warning_widget = widgets.HTML(
    value="""
    <div style="padding: 5px; background-color: #f8d7da; color: #721c24; border: 2px solid #f5c6cb; border-radius: 5px;">
        <strong>Warning!</strong> Select an option first.
    </div>
    """
)

In [109]:
error = widgets.HBox(
    [warning_widget],
    layout=widgets.Layout(
        width='auto',  
        height='auto',
        margin='auto',  
        align_items='flex-end',  
        justify_content='flex-end' 
    )
)

In [110]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = file_name + ".ipynb"
        js_code = f"window.open('{notebook_filename}', '_self');"
#         link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(Javascript(js_code))

back_button.on_click(on_back_button_clicked)

In [111]:
output = widgets.Output()
def on_button_clicked(event):
    with output:
        clear_output()

        if not selected:
            display(error)
        else:
            next_notebook = "num_cams"

            notebook_filename = next_notebook + ".ipynb"
            # JavaScript to open the URL
            js_code = f"window.open('{notebook_filename}', '_self');"

            # Storing current page name for later use when backtracking
            back_list.append("objective")
            with open (back_path, "w")as f:
                json.dump(back_list, f)

            display(Javascript(js_code))

submit_button.on_click(on_button_clicked)

In [113]:
# Creating containers for all the widgets along with their css properties

all_button = widgets.HBox(
    [back_button, submit_button],
    layout=widgets.Layout(
        width='100%',  
        height='auto',
#         border='solid 2px grey',
        padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='space-between' 
    )
)
buttons = widgets.VBox(
    [output, all_button],
    layout=widgets.Layout(
        width='100%',  
        height='auto',  
#         border='solid 2px grey',
        padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='center'  
    )
)



upper = widgets.VBox(
    [logical_integer_container, buttons],
    layout=widgets.Layout(
        width='95%',  
        height='auto', 
        border='solid 1px grey',
        padding='10px',
        margin='auto',  
        align_items='center',  
        justify_content='center' 
    )
)
upper.add_class("custom-vbox")

popup = widgets.VBox(children=[button, output_popup], layout=widgets.Layout(
        width='95%',  
        height='auto',  
#         border='solid 2px grey',
        padding='5px',
        margin='auto',  
        align_items='center', 
        justify_content='center'  
    ))
# [header_container, background_color_html, upper, footer, popup]
right_col = widgets.VBox(children=[header_container, background_color_html, upper, footer_output, popup], layout=widgets.Layout(
        width='88%',  #, popup
        height='100%',  
        border_left='solid 1px grey',
        padding='5px',
        margin='auto',  
        align_items='center',  
        justify_content='center'
))

cols = widgets.HBox(
    [left_col, right_col],
    layout=widgets.Layout(
        width='95%',  
        height='auto',  #90vh
        border='solid 2px grey',
#         padding='5px',
        margin='auto',  
        align_items='flex-start', 
        justify_content='center'
    )
)

display(cols)

In [15]:
### DEMO MARKDOWN CONTENT READING

# import ipywidgets as widgets

# # Create a GridSpecLayout
# grid = widgets.GridspecLayout(3, 3, height='300px')

# # Add markdown content as HTML
# html_content = widgets.HTML("""
#     <h1>Demo Markdown Content</h1>
#     <ul>
#         <li>This is a bullet point.</li>
#         <li>Another bullet point.</li>
#     </ul>
#     <h2>Code example</h2>
#     <pre><code>def hello_world():\n    print("Hello, World!")</code></pre>
#     <h3>Link</h3>
#     <a href="https://www.google.com">Google</a>
# """)

# # Add the HTML content to a specific cell in the grid
# grid[0, 0] = html_content

# # Display the grid
# display(grid)
